In [13]:
import pandas as pd

In [14]:
data = pd.read_csv('data/processed/bus_network_data.csv')
data = data.iloc[:,1:]

In [15]:
data.columns

Index([u'bus_line', u'direction', u'date', u'month', u'day', u'day_of_week',
       u'time_period', u'hour', u'minute', u'trip_time', u'avg_trip_time',
       u'std_trip_time', u'delay_time', u'delay', u'Conditions', u'Humidity',
       u'PrecipitationIn', u'TemperatureF', u'VisibilityMPH', u'Wind SpeedMPH',
       u'totalInjuries', u'pavementScore', u'potholeCount', u'prev_trip_ratio',
       u'ntwk_delay_lag1hr'],
      dtype='object')

In [16]:
data.head()

,bus_line,direction,date,month,day,day_of_week,time_period,hour,minute,trip_time,...,Humidity,PrecipitationIn,TemperatureF,VisibilityMPH,Wind SpeedMPH,totalInjuries,pavementScore,potholeCount,prev_trip_ratio,ntwk_delay_lag1hr
0,B11,2.0,2016-01-01,1,1,Friday,PeakAM,9,44,38.450000,...,58.0,0.0,39.9,10.0,10.4,162,2110.0,237,0.638492,2.0
1,B11,1.0,2016-01-01,1,1,Friday,PeakAM,9,52,53.900000,...,58.0,0.0,39.9,10.0,10.4,162,2110.0,237,1.055520,2.0
2,B11,2.0,2016-01-01,1,1,Friday,MidDay,10,10,34.700000,...,58.0,0.0,39.9,10.0,6.9,162,2110.0,237,1.017161,4.0
3,B11,1.0,2016-01-01,1,1,Friday,MidDay,10,23,41.050000,...,58.0,0.0,39.9,10.0,6.9,162,2110.0,237,0.979406,4.0
4,B11,2.0,2016-01-01,1,1,Friday,MidDay,10,33,40.183333,...,58.0,0.0,39.9,10.0,6.9,162,2110.0,237,0.591185,4.0


In [11]:
data.ntwk_delay_lag1hr.value_counts()

0.0    311752
5.0    209585
4.0    113779
3.0     67141
1.0     45926
2.0     37263
Name: ntwk_delay_lag1hr, dtype: int64

In [18]:
data.Conditions.value_counts()

Clear                  399485
Overcast               158753
Mostly Cloudy           59500
Partly Cloudy           48567
Scattered Clouds        33533
Light Rain              28833
Unknown                 27123
Haze                    16992
Rain                     7032
Light Snow               5003
Heavy Rain               2434
Snow                      908
Light Freezing Rain       345
Heavy Snow                113
Light Freezing Fog          1
Name: Conditions, dtype: int64

In [19]:
features = [u'bus_line', u'direction',
            #u'date',
            u'month',
            #u'day',
            u'day_of_week',
            u'time_period',
            u'hour',
            #u'minute',
            #u'trip_time', 
            #u'avg_trip_time',
            #u'std_trip_time', 
            #u'delay_time',
            u'delay', #this is our Y
            u'Conditions', 
            u'Humidity',
            u'PrecipitationIn',
            u'TemperatureF',
            u'VisibilityMPH',
            u'Wind SpeedMPH',
            u'totalInjuries', # cur for networks pd.cut(5)
            u'pavementScore', #
            u'potholeCount', # 
            u'prev_trip_ratio',
            u'ntwk_delay_lag1hr']

data = data.loc[:,features]
data.head()

,bus_line,direction,month,day_of_week,time_period,hour,delay,Conditions,Humidity,PrecipitationIn,TemperatureF,VisibilityMPH,Wind SpeedMPH,totalInjuries,pavementScore,potholeCount,prev_trip_ratio,ntwk_delay_lag1hr
0,B11,2.0,1,Friday,PeakAM,9,0,Overcast,58.0,0.0,39.9,10.0,10.4,162,2110.0,237,0.638492,2.0
1,B11,1.0,1,Friday,PeakAM,9,0,Overcast,58.0,0.0,39.9,10.0,10.4,162,2110.0,237,1.055520,2.0
2,B11,2.0,1,Friday,MidDay,10,0,Overcast,58.0,0.0,39.9,10.0,6.9,162,2110.0,237,1.017161,4.0
3,B11,1.0,1,Friday,MidDay,10,0,Overcast,58.0,0.0,39.9,10.0,6.9,162,2110.0,237,0.979406,4.0
4,B11,2.0,1,Friday,MidDay,10,0,Overcast,58.0,0.0,39.9,10.0,6.9,162,2110.0,237,0.591185,4.0


In [20]:
#naive bayes
#bayes networks